In [1]:
!which python
!python --version
!nvidia-smi

/Users/nadavt/opt/anaconda3/envs/jaymody-sps-env/bin/python
Python 3.9.10
zsh:1: command not found: nvidia-smi


In [2]:
# !python -m torch.utils.collect_env

In [3]:
pip install -q torch transformers

Note: you may need to restart the kernel to use updated packages.


# Generate via `transformers`

In [4]:
# Load model & Sanity check
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

draft_tokenizer = AutoTokenizer.from_pretrained("gpt2")
target_tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

draft_model = GPT2LMHeadModel.from_pretrained("gpt2")
target_model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

draft_inputs = draft_tokenizer("Hello, my dog is cute", return_tensors="pt")
target_inputs = target_tokenizer("Hello, my dog is cute", return_tensors="pt")
# Assert that `draft_tokenizer` and `target_tokenizer` are equivalent
assert torch.all(draft_inputs.input_ids == target_inputs.input_ids)
assert torch.all(draft_inputs.attention_mask == target_inputs.attention_mask)

draft_outputs = draft_model(**draft_inputs, labels=draft_inputs["input_ids"])
target_outputs = target_model(**target_inputs, labels=target_inputs["input_ids"])


loss = draft_outputs.loss
logits = draft_outputs.logits
logits.shape

/Users/nadavt/opt/anaconda3/envs/jaymody-sps-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 6, 50257])

In [5]:
# Generate
draft_outputs = draft_model.generate(draft_inputs.input_ids, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False)
target_outputs = target_model.generate(target_inputs.input_ids, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False)
print("draft_outputs.sequences == target_outputs.sequences:")
print(draft_outputs.sequences == target_outputs.sequences)

print("draft_outputs.sequences:")
print(draft_tokenizer.batch_decode(draft_outputs.sequences))

print("target_outputs.sequences:")
print(target_tokenizer.batch_decode(target_outputs.sequences))

print("draft_outputs:")
print(draft_outputs)
print("====")
transition_scores = draft_model.compute_transition_scores(
    draft_outputs.sequences, draft_outputs.scores, normalize_logits=True
)
print("transition_scores:")
print(transition_scores)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


draft_outputs.sequences == target_outputs.sequences:
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False]])
draft_outputs.sequences:
["Hello, my dog is cute. I'm not sure if she's a puppy or not. I'm not sure if she's"]
target_outputs.sequences:
["Hello, my dog is cute. I'm going to take him to the park today. I'm going to take him to the"]
draft_outputs:
GreedySearchDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  3290,   318, 13779,    13,   314,  1101,   407,
          1654,   611,   673,   338,   257, 26188,   393,   407,    13,   314,
          1101,   407,  1654,   611,   673,   338]]), scores=(tensor([[-77.4425, -80.4463, -88.0498,  ..., -96.2564, -93.6345, -84.0666]]), tensor([[-142.3193, -142.1797, -144.0388,  ..., -155.6977, -153.8761,
         -137.7942]]), tensor([[-175.1097, -174.3555, -180.2891,  ..., -186.55

In [6]:
K = 3
additional_input_ids = torch.arange(K).unsqueeze(0).repeat(3, 1)
print("additional_input_ids.shape:", additional_input_ids.shape)
print("additional_input_ids:")
additional_input_ids

additional_input_ids.shape: torch.Size([3, 3])
additional_input_ids:


tensor([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]])

In [7]:
torch.tile(draft_inputs.input_ids, (K, 1))

tensor([[15496,    11,   616,  3290,   318, 13779],
        [15496,    11,   616,  3290,   318, 13779],
        [15496,    11,   616,  3290,   318, 13779]])

In [8]:
# Verifying: generate(**inputs, ...) == generate(inputs.input_ids, ...)
print("Is attention_mask is unnecessary?", torch.all(draft_model.generate(draft_inputs.input_ids, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False).sequences == draft_model.generate(**draft_inputs, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False).sequences))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Is attention_mask is unnecessary? tensor(True)


In [9]:
from typing import Any
from torch import Tensor
from collections.abc import Iterable


def print_shapes(obj: Any, level: int = 0):
    padding = "--" * level
    if isinstance(obj, Tensor):
        print(f"{padding}{obj.shape}")
    elif isinstance(obj, dict):
        for k, v in obj.items():
            print(f"{padding}{k}")
            print_shapes(v, level + 1)
    elif isinstance(obj, str):
        print(f"{padding}type {type(obj)} is not supported")
    elif isinstance(obj, Iterable):
        for x in obj:
            print_shapes(x, level + 1)
    else:
        print(f"{padding}type {type(obj)} is not supported")


# # Test cases
# print_shapes(["test", Tensor([1, 2, 3]), {"key": Tensor([4, 5, 6])}])
# print("====")
# print_shapes("This is a string")
# print("====")
# print_shapes((1, 2, Tensor([7, 8, 9]), {"nested": [Tensor([10, 11, 12]), "string"]}))


In [10]:
import transformers

assert isinstance(draft_inputs, transformers.tokenization_utils_base.BatchEncoding)

In [11]:
print_shapes(draft_outputs)

sequences
--torch.Size([1, 26])
scores
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])


In [12]:
type(draft_outputs.scores), len(draft_outputs.scores)

(tuple, 20)

In [13]:
print(draft_outputs.scores[-1].shape)
print(draft_outputs.scores[-1].min())
print(draft_outputs.scores[-1].max())

torch.Size([1, 50257])
tensor(-156.0659)
tensor(-112.9710)


In [14]:
draft_tokenizer.batch_decode(draft_outputs.sequences)

["Hello, my dog is cute. I'm not sure if she's a puppy or not. I'm not sure if she's"]

# Ragged batch

In [15]:
pad_token_id = eos_token_id = torch.nan

In [16]:
import torch

print("Let's see how to create a batch of inputs with different lengths...", end="\n---\n")
print("Here is how the input_ids look like. It is a tensor of shape (batch_size=1, sequence_length):")
print(f"{draft_inputs.input_ids.shape=}")
print(f"{draft_inputs.input_ids=}")

print("---")
print("We want to create something similar... First, here is how we can concatenate new tokens to a sequence:")
# Initial sequence and new token
seq_len: int = 3
seq = torch.arange(seq_len, dtype=torch.float).view(1, -1)  # Shape: (batch_size=1, seq_len=3)
new_token = torch.Tensor([3]).view(1, -1)  # Shape: (batch_size=1, seq_len=1)
dft_seq = torch.cat((seq, new_token), dim=-1)  # Shape: (batch_size=1, seq_len=4)
print(f"{seq=}")
print(f"{seq.shape=}")
print()
print(f"{new_token=}")
print(f"{new_token.shape=}")
print()
print(f"{dft_seq=}")
print(f"{dft_seq.shape=}")

print("---")
print("=== NESTED TENSORS ===")
batch: Tensor = torch.nested.nested_tensor([seq.squeeze(), dft_seq.squeeze()])
batch_reversed: Tensor = torch.nested.nested_tensor([dft_seq.squeeze(), seq.squeeze()])
print(f"{batch=}")
print(f"{batch_reversed=}")

print("---")
print("Unfortunately, converting a nested tensor to a padded tensor is always left-padded:")
padded_batch: Tensor = torch.nested.to_padded_tensor(batch, padding=pad_token_id)
padded_batch_reversed: Tensor = torch.nested.to_padded_tensor(batch_reversed, padding=pad_token_id)
print(f"{padded_batch=}")
print(f"{padded_batch.shape=}")
print()
print(f"{padded_batch_reversed=}")
print(f"{padded_batch_reversed.shape=}")

print("---")
print("=== LEFT PADDING ===")
print("Padding the original sequence to the same length as new_seq:")
pad_size = dft_seq.size(1) - seq.size(1)
print(f"{pad_size=}")

padded_seq = torch.nn.functional.pad(seq, (pad_size, 0), value=pad_token_id)  # Left padding
print(f"{padded_seq=}")
print("Creating a batch with padded_seq and new_seq:")
padded_batch = torch.stack([padded_seq.squeeze(), dft_seq.squeeze()], dim=0)
print(f"{padded_batch=}")
print(f"{padded_batch.shape=}")

Let's see how to create a batch of inputs with different lengths...
---
Here is how the input_ids look like. It is a tensor of shape (batch_size=1, sequence_length):
draft_inputs.input_ids.shape=torch.Size([1, 6])
draft_inputs.input_ids=tensor([[15496,    11,   616,  3290,   318, 13779]])
---
We want to create something similar... First, here is how we can concatenate new tokens to a sequence:
seq=tensor([[0., 1., 2.]])
seq.shape=torch.Size([1, 3])

new_token=tensor([[3.]])
new_token.shape=torch.Size([1, 1])

dft_seq=tensor([[0., 1., 2., 3.]])
dft_seq.shape=torch.Size([1, 4])
---
=== NESTED TENSORS ===
batch=nested_tensor([
  tensor([0., 1., 2.]),
  tensor([0., 1., 2., 3.])
])
batch_reversed=nested_tensor([
  tensor([0., 1., 2., 3.]),
  tensor([0., 1., 2.])
])
---
Unfortunately, converting a nested tensor to a padded tensor is always left-padded:
padded_batch=tensor([[0., 1., 2., nan],
        [0., 1., 2., 3.]])
padded_batch.shape=torch.Size([2, 4])

padded_batch_reversed=tensor([[0., 

/var/folders/jq/b3gdpg7x2yg_rv2dwd3ly30w0000gp/T/ipykernel_7706/2356974417.py:26: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:180.)
  batch: Tensor = torch.nested.nested_tensor([seq.squeeze(), dft_seq.squeeze()])


In [17]:
# Initial sequence, K drafts, and new sequences
print("If we generate K drafts, the batch size will be K+1:")
K: int = 3
dft_seq: Tensor = torch.arange(10, 10 + K).view(1, -1)  # Shape: (batch_size=1, seq_len=K)
print(f"{dft_seq=}")
print(f"{dft_seq.shape=}")
dft_seq = torch.cat((seq, dft_seq), dim=-1)  # Shape: (batch_size=1, seq_len=seq_len+K)
print(f"{dft_seq=}")

print("---")
print("Let's create a batch with the original sequence and K drafts via left padding:")
# Initialize a list to hold the padded sequences
padded_batch = []

# Iteratively pad each sequence
for i in range(K + 1):
    # Calculate the amount of padding needed
    pad_size = dft_seq.size(1) - i - seq.size(1)
    
    # Create the padded sequence
    padded_seq = torch.cat([torch.full((1, pad_size), pad_token_id), seq], dim=-1)
    
    # Add the draft tokens as needed
    if i > 0:
        padded_seq = torch.cat([padded_seq, dft_seq[:, :i]], dim=-1)
    
    # Add the padded sequence to the list
    padded_batch.append(padded_seq)

# Combine all padded sequences into a batch
padded_batch = torch.cat(padded_batch, dim=0)

print(f"{padded_batch.shape=}")
print("padded_batch=")
print(padded_batch)

If we generate K drafts, the batch size will be K+1:
dft_seq=tensor([[10, 11, 12]])
dft_seq.shape=torch.Size([1, 3])
dft_seq=tensor([[ 0.,  1.,  2., 10., 11., 12.]])
---
Let's create a batch with the original sequence and K drafts via left padding:
padded_batch.shape=torch.Size([4, 6])
padded_batch=
tensor([[nan, nan, nan, 0., 1., 2.],
        [nan, nan, 0., 1., 2., 0.],
        [nan, 0., 1., 2., 0., 1.],
        [0., 1., 2., 0., 1., 2.]])


In [27]:
print("Optimize the above code by pre-allocating the padded_batch:")

def get_batch(
    dft_seq: Tensor,  # A sequence of input_ids of shape (batch_size=1, seq_len + K)
    K: int,
    pad_token_id,
) -> Tensor:
    """
    Returns a left-padded ragged batch of shape (K + 1, seq_len + K).
    """
    # Infer the length of the original sequence (seq_len)
    seq_len = dft_seq.size(1) - K

    # Pre-allocate a tensor filled with the pad_token_id for padded_batch
    padded_batch = torch.full((K + 1, seq_len + K), pad_token_id)

    # Extract the original sequence from dft_seq
    seq = dft_seq[:, :seq_len]

    # Fill in the appropriate values in the pre-allocated tensor
    for i in range(K + 1):
        start_idx = K - i
        padded_batch[i, start_idx: start_idx + seq_len] = seq
        if i > 0:
            padded_batch[i, start_idx + seq_len: start_idx + seq_len + i] = dft_seq[:, seq_len:seq_len + i]

    return padded_batch

print(f"{dft_seq=}")
print(f"{dft_seq.shape=}")
print(f"{K=}")
print(f"{pad_token_id=}")
padded_batch = get_batch(dft_seq=dft_seq, K=K, pad_token_id=pad_token_id)
padded_batch

Optimize the above code by pre-allocating the padded_batch:
dft_seq=tensor([[ 0.,  1.,  2., 10., 11., 12.]])
dft_seq.shape=torch.Size([1, 6])
K=3
pad_token_id=nan


tensor([[nan, nan, nan,  0.,  1.,  2.],
        [nan, nan,  0.,  1.,  2., 10.],
        [nan,  0.,  1.,  2., 10., 11.],
        [ 0.,  1.,  2., 10., 11., 12.]])

In [32]:
seq: Tensor = draft_inputs.input_ids
dft_seq: Tensor = draft_outputs.sequences
print(f"{seq=}")
print(f"{seq.shape=}")
print(f"{dft_seq=}")
print(f"{dft_seq.shape=}")

print("---")
pad_token_id: int = target_tokenizer.pad_token_id or target_tokenizer.eos_token_id
print(f"{pad_token_id=}")
K: int = dft_seq.size(1) - seq.size(1)
print(f"{K=}")
batch: Tensor = get_batch(dft_seq, K=K, pad_token_id=pad_token_id)
print(f"{batch.shape=}")
print("target_tokenizer.batch_decode(batch):")
batched_decoded: list[str] = target_tokenizer.batch_decode(batch)
batched_decoded

seq=tensor([[15496,    11,   616,  3290,   318, 13779]])
seq.shape=torch.Size([1, 6])
dft_seq=tensor([[15496,    11,   616,  3290,   318, 13779,    13,   314,  1101,   407,
          1654,   611,   673,   338,   257, 26188,   393,   407,    13,   314,
          1101,   407,  1654,   611,   673,   338]])
dft_seq.shape=torch.Size([1, 26])
---
pad_token_id=50256
K=20
batch.shape=torch.Size([21, 26])
target_tokenizer.batch_decode(batch):


['<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>Hello, my dog is cute',
 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>Hello, my dog is cute.',
 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>Hello, my dog is cute. I',
 "<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endo